# Playbook Explorer

This notebook is intended to be a live example of how to work with SysML v2 models at analysis-time. For these purposes, the following terms are introduced:
* An *interpretation* is the mapping of user model symbols (the "M1 model" in OMG-speak) into semantically-correct symbols that represent real world objects meant to conform to the model (the "M0" in OMG-speak). Interpretation semantics are inspired by https://www.w3.org/TR/owl2-direct-semantics/ and are mostly similar.
* A *sequence* for an interpretation contains *atoms* or *instances* that match to real world things. Reading a sequence from left to right provides a set of nested contexts for the atoms that is important to the interpretation. For example [Rocket#0, LS#3] is a 2-sequence to describe facts around the LS#3 atom when it is considered in context for Rocket#0. This is an important idea for the SysML time and occurrence model where one may want to see how values change under different conditions.

This is a notebook that walks through the random interpretation generator to help developers working on their own interpreters.

## Example Model

The model that is used for this example is the SysML v2 Kerbal model written by Bjorn Cole

The textual SysML v2 model is the Kerbal model:

    package Kerbal {
        package 'Rocket Building' {
            import ScalarFunctions::*;

            part def Rocket {
                part stages : 'Rocket Stage' [1..5] {
                    // placing this here because previous stages only make sense in context of a full vehicle
                    ref 'Carried Stage' : 'Rocket Stage' [1];
                    attribute 'Payload Mass' : Real;
                    attribute 'Loaded Mass' : Real;
                    attribute 'Burnout Mass' : Real;
                    part 'Coupler to Carrying Stage' : Coupler [0..8] {
                        attribute 'Separation Force' : Real;
                    }
                }
            }
            item def Oxidizer;
            item def Fuel;
            item def 'Solid Propellant';

            // TODO: Something something part symmetry
            abstract part def 'Rocket Stage';

            // use stage types to enforce matching
            part def 'Liquid Stage' :> 'Rocket Stage' {
                part engines : 'Liquid Engine' [0..8];
                part tanks : 'Fuel Tank Section' [0..30];
                attribute 'Full Mass' : Real = sum(engines->collect p:'Kerbal Rocket Part' (p::Mass)) +
                                        sum(tanks->collect p:'Fuel Tank Section' (p::'Full Mass'));

                attribute 'Empty Mass' : Real;
            }
            part def 'Solid Stage' :> 'Rocket Stage' {
                part boosters : 'Solid Booster' [0..8];
                attribute 'Full Mass' : Real;
                attribute 'Empty Mass' : Real;
            }

            part def 'Coupler' :> 'Kerbal Rocket Part';

            abstract part def 'Fuel Tank Section' :> 'Kerbal Rocket Part' {
                attribute 'Liquid Fuel' : Real;
                attribute 'Oxidizer' : Real;
                attribute 'Full Mass' : Real;
                attribute 'Empty Mass' : Real;
            }

            abstract part def 'Liquid Engine' :> 'Kerbal Rocket Part' {
                attribute 'Specific Impulse' : Real;
                attribute 'Thrust' : Real;
            }
            abstract part def 'Solid Booster' :> 'Kerbal Rocket Part' {
                attribute 'Specific Impulse' : Real;
                attribute 'Full Mass' : Real;
                attribute 'Empty Mass' : Real;
                attribute 'Thrust' : Real;
            }

            part def 'Pod' :> 'Kerbal Rocket Part'  {
                attribute Torque : Real;
            }
            part def 'Parachute' :> 'Kerbal Rocket Part';

            part def 'Kerbal Rocket Part' {
                attribute Mass : Real;
                attribute 'Max Temperature' : Real;
            }
        }
        package 'Parts Library' {
            import ScalarFunctions::*;
            part def 'FL-T200 Fuel Tank' :> 'Rocket Building'::'Fuel Tank Section' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Full Mass' = 1.125;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Empty Mass' = 0.125;
            }
            part def 'FL-T100 Fuel Tank' :> 'Rocket Building'::'Fuel Tank Section' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Full Mass' = 0.5625;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Empty Mass' = 0.0625;
            }
            part def 'Mk1 Command Pod' :> 'Rocket Building'::'Pod';
            part def 'LV-T45 "Swivel" Liquid Fuel Engine' :> 'Rocket Building'::'Liquid Engine' {
                attribute 'Specific Impulse' : Real :>> 'Rocket Building'::'Liquid Engine'::'Specific Impulse' = 170.0;
                attribute 'Thrust' : Real :>> 'Rocket Building'::'Liquid Engine'::'Thrust' = 167.97;
                attribute 'Mass' : Real :>> 'Rocket Building'::'Kerbal Rocket Part'::Mass = 1.50;
            }
            part def 'RT-5 "Flea" Solid Fuel Booster' :> 'Rocket Building'::'Solid Booster' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Full Mass' = 1.50;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Empty Mass' = 0.45;
                attribute 'Specific Impulse' : Real :>> 'Rocket Building'::'Solid Booster'::'Specific Impulse' = 140.0;
                attribute 'Thrust' : Real :>> 'Rocket Building'::'Solid Booster'::'Thrust' = 162.91;
            }
            part def 'RT-10 "Hammer" Solid Fuel Booster' :> 'Rocket Building'::'Solid Booster' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Full Mass' = 3.56;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Empty Mass' = 0.75;
                attribute 'Specific Impulse' : Real :>> 'Rocket Building'::'Solid Booster'::'Specific Impulse' = 170.0;
                attribute 'Thrust' : Real :>> 'Rocket Building'::'Solid Booster'::Thrust = 197.90;
            }
        }
    }

## Imports

Import key modules, functions, and classes from the PyMBE library:

In [1]:
import os
from pathlib import Path

import networkx as nx

import pymbe.api as pm
from pymbe.client import SysML2Client
from pymbe.label import *
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.calc_dependencies import generate_execution_order, generate_parameter_signature_map
from pymbe.interpretation.interp_playbooks import *
from pymbe.interpretation.results import *

## Key IDs

The unique identifiers below are useful references for walking through the interpretations generated in this notebook.

In [2]:
fts_full_mass = 'b97614b5-a413-471c-ade2-e1bfc7816543' # Full Mass Attribute under Fuel Tank Sections
ft200_full_mass = 'e6d1c157-fbff-4b45-9a8a-ed192431510f' # Full Mass Attribute under FL-T200 Fuel Tank
ft100_full_mass = 'f7d4ac57-9a58-4bec-9153-2254019de47d' # Full Mass Attribute under FL-T100 Fuel Tank
liquid_stage_full_mass = 'd2ccb1d4-015f-4d74-82ee-b795c2ea1c16' # Full Mass Attribute under Liquid Stage
top_plus = '3b2d0703-6c19-4584-901b-9aabb156fde2' # The '+' Expression under the Full Mass Attribute under Liquid Stage
tank_mass_sum_1 = '75ea8b2b-8805-45df-bac1-5ffd955e4982' # The 'sum' Expression that sums Full Mass of tanks
#collect_1 = 'd6644a0a-6eef-49c1-a770-60886073554c' # The 'collect' Expression that gathers Full Masses that apply to the current scope
#collect_1_result = '2caccce7-a0b4-4926-8f24-0dbffb92f6ad' # The result parameter of above collect expression
full_mass_dot = '86a423f7-18b6-4a86-ac52-2f69302121ab' # Expression that scopes Full Mass attribute in FeatureReferenceExpression
fre_1 = '6505ffab-edc1-442d-9d99-91802d764df8' # Expression to point to the instances of Full Mass
fre_1_result = 'b316c054-39af-4c78-a9f8-4e0445dfb836' # Result of gathering all instances of Full Mass from the interpretation
sum_1_result = '5f2e43c7-eada-4be7-b934-ed30180b2aec' # Result of the sum Expression on Full Mass
x = '85e06150-af83-4262-81c4-c95aa2817ce7'

real = 'ee569a49-1a15-49e8-990c-6cd545b2971e'

## Client Setup

The example here uses a local copy of the JSON file obtained by a GET operation on the SysML v2 API at:
http://sysml2-sst.intercax.com:9000/projects/a4f6a618-e4eb-4ac8-84b8-d6bcd3badcec/commits/c48aea9b-42fb-49b3-9a3e-9c39385408d7/elements?page[size]=5000

Create the client and load local data.

In [3]:
helper_client = SysML2Client()

elements_data_path = Path("..") / "tests" / "fixtures" / "Kerbal.json"
assert elements_data_path.exists(), f"Could not find: '{elements_data_path}'"

helper_client._load_from_file(elements_data_path)

Create a graph representation of the model and load it into memory.

In [4]:
lpg = SysML2LabeledPropertyGraph()
lpg.update(helper_client.elements_by_id, False)

This is just a helper to make abbreviations more legible.

In [5]:
shorten_pre_bake = {
    'RT-10 "Hammer" Solid Fuel Booster': "RT-10",
    'RT-5 "Flea" Solid Fuel Booster': "RT-5",
    'LV-T45 "Swivel" Liquid Fuel Engine': "LV-T45",
    'FL-T100 Fuel Tank': "FL-T100",
    'FL-T200 Fuel Tank': "FL-T200"
}

## Understanding Expressions

Eventually this will move out to its own notebook ...

Expressions form trees inside the M1 model. Some work has to be done in order to understand them and how data should flow from result parameters in one expression to the input parameters of another.

Create an interpretation of the Kerbal model using the random generator playbook. In general, this randomly selects:
- The ratios of partitioning abstract classifier sequence sets into concrete sets. For example, one draw may choose 2 liquid stages and 3 solids.
- The number of sequences to create for a given feature multiplicity. For example, draw 2 for a 0..8 engines : Liquid Engine PartUsage.

The playbook also attempts to make sequences created obey the Subsetting relationship (elements marked with subsets in M1 model should have their interpretation sequences entirely included within the interpretation sequences of the superset).

In [6]:
exec_order = generate_execution_order(lpg)

These edge types are not in the graph: {'ResultExpressionMembership'}.
These edge types are not in the graph: {'ImpliedParameterFeedforward', 'ImpliedPathArgumentFeedforward', 'ImpliedReferentFeed'}.


c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:386: UserWarning: 'ImpliedPathArgumentFeedforward' is not a valid edge generator!
  warn(f"'{implied_edge_type}' is not a valid edge generator!")
c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:386: UserWarning: 'ImpliedReferentFeed' is not a valid edge generator!
  warn(f"'{implied_edge_type}' is not a valid edge generator!")


In [7]:
pprint_edges(exec_order, helper_client.elements_by_id)

[['96ca6e7a-9a9c-4eba-9529-e4d5ac29118a «LiteralInteger»',
  '$result',
  'Output'],
 ['167.97 «Occurred LiteralReal»', '$result', 'Output'],
 ['$result', 'Thrust: Real', 'ValueBinding'],
 ['Thrust: Real', 'Thrust: Real', 'Redefinition'],
 ['bb2d418b-3cce-4cd2-b214-d3a89f25df31 «LiteralInteger»',
  '$result',
  'Output'],
 ['3.56 «Occurred LiteralReal»', '$result', 'Output'],
 ['1.5 «Occurred LiteralReal»', '$result', 'Output'],
 ['$result', 'Full Mass: Real', 'ValueBinding'],
 ['$result', 'Full Mass: Real', 'ValueBinding'],
 ['Full Mass: Real', 'Full Mass: Real', 'Redefinition'],
 ['Full Mass: Real', 'Full Mass: Real', 'Redefinition'],
 ['5 «Occurred LiteralInteger»', '$result', 'Output'],
 ['6255fb87-8ced-4a3f-8359-4afab73e6606 «LiteralInteger»',
  '$result',
  'Output'],
 ['170.0 «Occurred LiteralReal»', '$result', 'Output'],
 ['$result', 'Specific Impulse: Real', 'ValueBinding'],
 ['Specific Impulse: Real', 'Specific Impulse: Real', 'Redefinition'],
 ['1 «Occurred LiteralInteger»',

In [8]:
sig_map = generate_parameter_signature_map(helper_client.elements_by_id, exec_order)
sig_map

{'faca6d46-8bd7-4741-90f5-135944589e67': '96ca6e7a-9a9c-4eba-9529-e4d5ac29118a «LiteralInteger»',
 'd79192d7-bf71-49c8-b671-dab115275f7d': '167.97 «Occurred LiteralReal»',
 'e412656d-eea2-4650-8b87-6d66460e5010': 'bb2d418b-3cce-4cd2-b214-d3a89f25df31 «LiteralInteger»',
 '2759f68a-b09e-44ff-9b45-c4ba28314ecc': '3.56 «Occurred LiteralReal»',
 'dcc52f93-c89b-4b62-a94e-c50035648450': '1.5 «Occurred LiteralReal»',
 '4a866edb-9d50-454d-a574-f60142e81399': '5 «Occurred LiteralInteger»',
 '715e7c98-1d48-49ce-bf0e-49135ae2d97d': '6255fb87-8ced-4a3f-8359-4afab73e6606 «LiteralInteger»',
 'a7488413-69b7-4581-8a63-70faa4540ffa': '170.0 «Occurred LiteralReal»',
 'aff8a5d1-1560-4ce3-b001-39da2b3ef833': '1 «Occurred LiteralInteger»',
 '5979ec43-005d-4d53-b7ba-df5baf05126d': 'c6f72048-3c8c-4c5a-bdd0-f902736315e9 «LiteralInteger»',
 '5351a395-e304-4345-a7e0-5ac66c874375': '8 «Occurred LiteralInteger»',
 '6a514fa1-130d-4048-bb18-0bb6191e503e': '30 «Occurred LiteralInteger»',
 'cc0f5b38-e495-4836-9b21-980

In [9]:
get_qualified_label(
    helper_client.elements_by_id["4eb53d3c-c93d-4900-9885-6ec99150dc01"],
    helper_client.elements_by_id,
    sig_map
)

'Model::Kerbal::Rocket Building::Liquid Stage::Full Mass: Real::+ (sum (engines.Mass (FRE.engines)), sum (tanks.Full Mass (FRE.tanks))) => $result::sum (engines.Mass (FRE.engines)) => $result::engines.Mass (FRE.engines) => $result::engines.Mass (FRE.engines)'

In [10]:
get_qualified_label(
    helper_client.elements_by_id["e5d649cf-b6cf-4426-b778-516a2d900029"],
    helper_client.elements_by_id,
    sig_map
)

'Model::Kerbal::Rocket Building::Liquid Stage::Full Mass: Real::+ (sum (engines.Mass (FRE.engines)), sum (tanks.Full Mass (FRE.tanks))) => $result::sum (engines.Mass (FRE.engines)) => $result::engines.Mass (FRE.engines)'

In [11]:
m0_interpretation = random_generator_playbook(
    helper_client.elements_by_id,
    lpg,
    shorten_pre_bake,
)

In [12]:
pprint_single_id_list(list(m0_interpretation.keys()), helper_client.elements_by_id)

['Real',
 'LV-T45 "Swivel" Liquid Fuel Engine',
 'Coupler',
 'RT-10 "Hammer" Solid Fuel Booster',
 'RT-5 "Flea" Solid Fuel Booster',
 'Solid Stage',
 'Liquid Stage',
 'FL-T100 Fuel Tank',
 'FL-T200 Fuel Tank',
 'Mk1 Command Pod',
 'Parachute',
 'Liquid Engine',
 'Fuel Tank Section',
 'Solid Booster',
 'Pod',
 'Kerbal Rocket Part',
 'Rocket Stage',
 'Rocket',
 'Empty Mass: Real',
 'Liquid Fuel: Real',
 'Full Mass: Real',
 'Oxidizer: Real',
 'Full Mass: Real',
 'Empty Mass: Real',
 'boosters: Solid Booster',
 'Specific Impulse: Real',
 'Thrust: Real',
 'Empty Mass: Real',
 'Full Mass: Real',
 'stages: Rocket Stage',
 'Burnout Mass: Real',
 'Loaded Mass: Real',
 'Payload Mass: Real',
 'Coupler to Carrying Stage: Coupler',
 'Separation Force: Real',
 'Empty Mass: Real',
 'Full Mass: Real',
 'Specific Impulse: Real',
 'Thrust: Real',
 'tanks: Fuel Tank Section',
 'engines: Liquid Engine',
 'Full Mass: Real',
 'Empty Mass: Real',
 'Empty Mass: Real',
 'Full Mass: Real',
 'Max Temperature: Re

To see how sequences are structured, the cell below renders sequences that show what type of atoms will fill particular positions in the sequence, as well as the maximum multiplicity (number of) sequences.

In [13]:
from pymbe.query.query import roll_up_upper_multiplicity, roll_up_multiplicity_for_type

feat_sequences = build_sequence_templates(lpg=lpg)

total = 0
for seq in feat_sequences:
    print(str(pprint_single_id_list(seq, lpg.nodes)) + ", " + str(roll_up_upper_multiplicity(lpg, lpg.nodes[seq[-1]])))

['Fuel Tank Section', 'Empty Mass: Real'], 151
['Fuel Tank Section', 'Liquid Fuel: Real'], 151
['Fuel Tank Section', 'Full Mass: Real'], 151
['Fuel Tank Section', 'Oxidizer: Real'], 151
['Solid Stage', 'Full Mass: Real'], 5
['Solid Stage', 'Empty Mass: Real'], 5
['Solid Stage', 'boosters: Solid Booster'], 40
['RT-10 "Hammer" Solid Fuel Booster', 'Specific Impulse: Real'], 41
['RT-10 "Hammer" Solid Fuel Booster', 'Thrust: Real'], 41
['RT-10 "Hammer" Solid Fuel Booster', 'Empty Mass: Real'], 41
['RT-10 "Hammer" Solid Fuel Booster', 'Full Mass: Real'], 41
['Rocket', 'stages: Rocket Stage', 'Burnout Mass: Real'], 5
['Rocket', 'stages: Rocket Stage', 'Loaded Mass: Real'], 5
['Rocket', 'stages: Rocket Stage', 'Payload Mass: Real'], 5
['Rocket', 'stages: Rocket Stage', 'Coupler to Carrying Stage: Coupler', 'Separation Force: Real'], 40
['FL-T200 Fuel Tank', 'Empty Mass: Real'], 151
['FL-T200 Fuel Tank', 'Full Mass: Real'], 151
['Liquid Engine', 'Specific Impulse: Real'], 41
['Liquid Engine', 

Once the interpretations are generated, we can look for expressions and create an execution order (this is similar to Excel builds a dependency graph internally to accelerate computations and partial updates when a user changes a cell value).

In [14]:
dcg = generate_execution_order(lpg)

One of the core tools in examining and working with the M1 model is using the get_projection function on the master graph to select out the kind of nodes and edges that will support other queries (roll-up using breadth-first search in reverse order, paths from one node to another to lay out sequences, etc.)

In [15]:
from pymbe.graph.calc_lpg import CalculationGroup
cg = CalculationGroup(lpg.get_projection("Expression Inferred"), m0_interpretation, dcg)

Display the calculation order determined by the algorithm, as well as a hint about what the step is (generating output from a function, moving values from function parameters to an attribute, applying redefinition, etc.)

In [16]:
for item in dcg:
    instance_safe = True
    if item[0] not in m0_interpretation or len(m0_interpretation[item[0]]) == 0:
        print("No instances for " + lpg.nodes[item[0]]['qualifiedName'])
        instance_safe = False
    if item[1] not in m0_interpretation or len(m0_interpretation[item[1]]) == 0:
        print("No instances for " + lpg.nodes[item[1]]['qualifiedName'])
        instance_safe = False
    if instance_safe:
        rep_source = m0_interpretation[item[0]][0][-1]
        rep_target = m0_interpretation[item[1]][0][-1]
        if len(item) == 3:
            safe_item = item[2]
        else:
            safe_item = 'None'
        print('(' + str(rep_source) + ', ' + str(rep_target) + ', ' + safe_item + ')')

(96ca6e7a-9a9c-4eba-9529-e4d5ac29118a «LiteralInteger», LS#0.96ca6e7a-9a9c-4eba-9529-e4d5ac29118a «LiteralInteger».$result#0 (unset), Output)
(167.97 «Occurred LiteralReal», LV-T45#0.167.97 «Occurred LiteralReal».$result#0 (unset), Output)
(LV-T45#0.167.97 «Occurred LiteralReal».$result#0 (unset), .Real#952 (unset), ValueBinding)
(.Real#952 (unset), .Real#2725 (unset), Redefinition)
No instances for Kerbal::'Rocket Building'::'Solid Stage'::boosters::null::null
No instances for Kerbal::'Rocket Building'::'Solid Stage'::boosters::null::null::'$result'
(3.56 «Occurred LiteralReal», RT-10#0.3.56 «Occurred LiteralReal».$result#0 (unset), Output)
(1.5 «Occurred LiteralReal», RT-5#0.1.5 «Occurred LiteralReal».$result#0 (unset), Output)
(RT-10#0.3.56 «Occurred LiteralReal».$result#0 (unset), .Real#1842 (unset), ValueBinding)
(RT-5#0.1.5 «Occurred LiteralReal».$result#0 (unset), .Real#737 (unset), ValueBinding)
(.Real#1842 (unset), .Real#1167 (unset), Redefinition)
(.Real#737 (unset), .Real#11

Use the calculation order in order to resolve the "unset" fields on many attributes to values where they are determined in the M1 model (e.g., using the ' = ' operator to assign values directly or connect to equations or analyses).

In [17]:
cg.solve_graph(lpg)

In [18]:
[pse for pse in cg.calculation_log if pse.startswith("[PSE]")]

['[PSE] Calling collect with base = LS#0, collection input LS#0.+ ($x, $y) => $result.sum ($collection) => $result.engines.Mass ($collection) => $result.$collection#0 ([[LS#0, LV-T45#16], [LS#0, LV-T45#19], [LS#1, LV-T45#22], [LS#1, LV-T45#11], [LS#1, LV-T45#9], [LS#1, LV-T45#5], [LS#2, LV-T45#10], [LS#2, LV-T45#7], [LS#2, LV-T45#38], [LS#2, LV-T45#30], [LS#2, LV-T45#14], [LS#2, LV-T45#37], [LS#2, LV-T45#33], [LS#2, LV-T45#1], [LS#3, LV-T45#18], [LS#3, LV-T45#29], [LS#3, LV-T45#21], [LS#3, LV-T45#3], [LS#3, LV-T45#24], [LS#3, LV-T45#2], [LS#3, LV-T45#17], [LS#4, LV-T45#23]]), and path input LS#0.+ ($x, $y) => $result.sum ($collection) => $result.engines.Mass ($collection) => $result.FRE.Mass.$result#0 ([[MCP#0, .Real#2857 (unset)], [Coupler#0, .Real#1247 (unset)], [Coupler#1, .Real#2747 (unset)], [Coupler#2, .Real#1109 (unset)], [Coupler#3, .Real#433 (unset)], [Coupler#4, .Real#974 (unset)], [Coupler#5, .Real#1009 (unset)], [Coupler#6, .Real#1867 (unset)], [Coupler#7, .Real#309 (unset)

## Calculation Results Shown

The following cells are a series of displays of relevant features in the interpretation.

In [19]:
m0_interpretation[sum_1_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.$result#0 (13.125)],
 [LS#1,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#1.+ ($x, $y) => $result.sum ($collection) => $result.$result#1 (24.75)],
 [LS#2,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#2.+ ($x, $y) => $result.sum ($collection) => $result.$result#2 (23.4375)],
 [LS#3,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#3.+ ($x, $y) => $result.sum ($collection) => $result.$result#3 (18.0)],
 [LS#4,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#4.+ ($x, $y) => $result.sum ($collection) => $result.$result#4 (7.3125)]]

In [20]:
m0_interpretation[x]

[[LS#0, + ($x, $y) => $result, LS#0.+ ($x, $y) => $result.$x#0 (3.0)],
 [LS#1, + ($x, $y) => $result, LS#1.+ ($x, $y) => $result.$x#1 (6.0)],
 [LS#2, + ($x, $y) => $result, LS#2.+ ($x, $y) => $result.$x#2 (12.0)],
 [LS#3, + ($x, $y) => $result, LS#3.+ ($x, $y) => $result.$x#3 (10.125)],
 [LS#4, + ($x, $y) => $result, LS#4.+ ($x, $y) => $result.$x#4 (1.125)]]

In [21]:
m0_interpretation[liquid_stage_full_mass]

[[LS#0, .Real#844 (16.125)],
 [LS#1, .Real#2383 (30.75)],
 [LS#2, .Real#787 (35.4375)],
 [LS#3, .Real#856 (28.125)],
 [LS#4, .Real#1462 (0.125)]]

In [22]:
m0_interpretation[fre_1_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  tanks.Full Mass ($collection) => $result,
  FRE.Full Mass,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.tanks.Full Mass ($collection) => $result.FRE.Full Mass.$result#0 ([[FL-T100#0, .Real#1668 (0.5625)], [FL-T100#1, .Real#2590 (0.5625)], [FL-T100#2, .Real#1617 (0.5625)], [FL-T100#3, .Real#721 (0.5625)], [FL-T100#4, .Real#1988 (0.125)], [FL-T100#5, .Real#1115 (0.5625)], [FL-T100#6, .Real#981 (0.5625)], [FL-T100#7, .Real#775 (0.5625)], [FL-T100#8, .Real#402 (0.5625)], [FL-T100#9, .Real#1866 (0.5625)], [FL-T100#10, .Real#904 (0.5625)], [FL-T100#11, .Real#1975 (0.5625)], [FL-T100#12, .Real#1631 (0.5625)], [FL-T100#13, .Real#1508 (0.5625)], [FL-T100#14, .Real#2065 (0.5625)], [FL-T100#15, .Real#2537 (0.5625)], [FL-T100#16, .Real#2420 (0.5625)], [FL-T100#17, .Real#3034 (0.5625)], [FL-T100#18, .Real#2992 (1.5)], [FL-T100#19, .Real#209 (0.5625)], [FL-T100#20, .Real#1773 (0.5625)], [FL-T100#21, .Real#13 (0.5625)], [

In [23]:
m0_interpretation[real]

[[.Real#0 (unset)],
 [.Real#1 (unset)],
 [.Real#2 (unset)],
 [.Real#3 (unset)],
 [.Real#4 (unset)],
 [.Real#5 (unset)],
 [.Real#6 (170.0)],
 [.Real#7 (unset)],
 [.Real#8 (unset)],
 [.Real#9 (unset)],
 [.Real#10 (197.9)],
 [.Real#11 (0.125)],
 [.Real#12 (0.0625)],
 [.Real#13 (0.5625)],
 [.Real#14 (0.125)],
 [.Real#15 (unset)],
 [.Real#16 (unset)],
 [.Real#17 (unset)],
 [.Real#18 (0.125)],
 [.Real#19 (170.0)],
 [.Real#20 (unset)],
 [.Real#21 (unset)],
 [.Real#22 (unset)],
 [.Real#23 (unset)],
 [.Real#24 (1.125)],
 [.Real#25 (0.5625)],
 [.Real#26 (unset)],
 [.Real#27 (unset)],
 [.Real#28 (unset)],
 [.Real#29 (unset)],
 [.Real#30 (1.125)],
 [.Real#31 (unset)],
 [.Real#32 (unset)],
 [.Real#33 (unset)],
 [.Real#34 (unset)],
 [.Real#35 (unset)],
 [.Real#36 (unset)],
 [.Real#37 (unset)],
 [.Real#38 (unset)],
 [.Real#39 (197.9)],
 [.Real#40 (0.45)],
 [.Real#41 (0.5625)],
 [.Real#42 (1.5)],
 [.Real#43 (unset)],
 [.Real#44 (unset)],
 [.Real#45 (unset)],
 [.Real#46 (unset)],
 [.Real#47 (1.125)],
 

Show all interpretation sequence sets (limited to length of 5).

In [24]:
for print_line in pprint_interpretation(m0_interpretation, helper_client.elements_by_id):
    print(print_line)

Real, id = ee569a49-1a15-49e8-990c-6cd545b2971e, size = 3038
[.Real#0 (unset)]
[.Real#1 (unset)]
[.Real#2 (unset)]
[.Real#3 (unset)]
[.Real#4 (unset)]
['..']
LV-T45 "Swivel" Liquid Fuel Engine, id = fb43246f-5c88-4d28-9fc9-e6db5e99ddd3, size = 40
[LV-T45#0]
[LV-T45#1]
[LV-T45#2]
[LV-T45#3]
[LV-T45#4]
['..']
Coupler, id = 41c3038a-b8d3-40c2-a7d3-4eda423cf4ed, size = 40
[Coupler#0]
[Coupler#1]
[Coupler#2]
[Coupler#3]
[Coupler#4]
['..']
RT-10 "Hammer" Solid Fuel Booster, id = ebc86331-ffdd-487d-8875-93803db7239d, size = 14
[RT-10#0]
[RT-10#1]
[RT-10#2]
[RT-10#3]
[RT-10#4]
['..']
RT-5 "Flea" Solid Fuel Booster, id = f8eab204-c3dd-4990-a5b0-0f0cf3659c84, size = 26
[RT-5#0]
[RT-5#1]
[RT-5#2]
[RT-5#3]
[RT-5#4]
['..']
Solid Stage, id = 5afd292a-dc86-4d57-9e5b-a14cabf9d67c, size = 0
Liquid Stage, id = 75498f95-93a7-4bc4-b3bf-644a08f95c5a, size = 5
[LS#0]
[LS#1]
[LS#2]
[LS#3]
[LS#4]
['..']
FL-T100 Fuel Tank, id = ab2b79e5-7ade-45c4-a3a7-ddf589ffc5e1, size = 45
[FL-T100#0]
[FL-T100#1]
[FL-T100#2]